In [59]:
from transformers import RobertaConfig, RobertaModel, RobertaModel, RobertaTokenizer
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sentence_transformers import SentenceTransformer, util
import glob
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
from sklearn.cluster import KMeans

In [41]:
script_AD=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\Data\AD"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df=pd.read_csv(f)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    script_AD.append(df)


df_AD=pd.concat(script_AD,axis=0,ignore_index=True)
label_AD=0
df_AD['label']=label_AD
df_AD


,sentence,label
0,mhm .,0
1,alright .,0
2,there's a young boy that's getting a cookie jar .,0
3,and he's in bad shape because the thing is fal...,0
4,and in the picture the mother is washin(g) dis...,0
...,...,...
1369,some of the dishes aren't washed yet .,0
1370,kids are takin(g) the cookies .,0
1371,he's gonna fall off of the stool +//.,0
1372,"no , what do you call that ?",0


In [42]:
script_HC=[]
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\Data\HC"
csv_files=glob.glob(path+'/*.csv')
for f in csv_files:
    df=pd.read_csv(f)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    script_HC.append(df)


df_HC=pd.concat(script_HC,axis=0,ignore_index=True)
label_HC=1
df_HC['label']=label_HC
df_HC

,sentence,label
0,the scene is in the kitchen .,1
1,the mother is wiping dishes and the water is r...,1
2,a boy is tryin(g) to get cookies out of a jar ...,1
3,the little girl is reacting to his falling .,1
4,it seems to be summer out .,1
...,...,...
1238,and the stool is tipping over .,1
1239,and he's sort_of put down the plates .,1
1240,and she's reaching up to get it but I don't se...,1
1241,yeah that's it .,1


In [43]:
df=pd.concat([df_AD,df_HC], ignore_index=True)
df

,sentence,label
0,mhm .,0
1,alright .,0
2,there's a young boy that's getting a cookie jar .,0
3,and he's in bad shape because the thing is fal...,0
4,and in the picture the mother is washin(g) dis...,0
...,...,...
2612,and the stool is tipping over .,1
2613,and he's sort_of put down the plates .,1
2614,and she's reaching up to get it but I don't se...,1
2615,yeah that's it .,1


In [44]:
roberta= SentenceTransformer('sentence-transformers/stsb-roberta-base-v2')
word_encodding=roberta.encode(df['sentence'])

In [70]:
word_encodding.shape
x=word_encodding
y=df['label']
print(x.dtype)

float32


In [74]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [84]:
model=KMeans(n_clusters=2,n_init=30,max_iter=1000,algorithm='elkan')
model.fit(x)

KMeans(algorithm='elkan', max_iter=1000, n_clusters=2, n_init=30)

In [115]:
df_zero_cluster=df[model.labels_==0]
df_first_cluster=df[model.labels_==1]
df_pred=pd.concat([df_first_cluster,df_zero_cluster],ignore_index=False)
df_pred


,sentence,label
4,and in the picture the mother is washin(g) dis...,0
5,and so the water is overflowing in the sink .,0
6,and the dishes might fall over there if you do...,0
9,but the water is still flowing .,0
16,and the sink is overflowing .,0
...,...,...
2612,and the stool is tipping over .,1
2613,and he's sort_of put down the plates .,1
2614,and she's reaching up to get it but I don't se...,1
2615,yeah that's it .,1


In [116]:
metrics.accuracy_score(df['label'],df_pred['label'])

0.7554451662208636

In [117]:
cosine_similarity = util.cos_sim(word_encodding, word_encodding)
cosine_similarity

tensor([[ 1.0000,  0.4614,  0.0843,  ...,  0.1456,  0.2213,  0.1391],
        [ 0.4614,  1.0000,  0.1108,  ...,  0.0491,  0.3583,  0.0535],
        [ 0.0843,  0.1108,  1.0000,  ...,  0.1165, -0.0585,  0.0172],
        ...,
        [ 0.1456,  0.0491,  0.1165,  ...,  1.0000,  0.0258,  0.1787],
        [ 0.2213,  0.3583, -0.0585,  ...,  0.0258,  1.0000,  0.0596],
        [ 0.1391,  0.0535,  0.0172,  ...,  0.1787,  0.0596,  1.0000]])